In [2]:
import re
import unicodedata
import os
os.chdir("D:/uppsala/15. ds projects/data")

def clean_raw_text(text):
    """
     OCR ：
    - （ﬁ, ﬂ ）
    - 、、
    -  ° → .， ... / . . . → .
    -  ASCII 
    """
    # 1️⃣ Unicode ：
    text = unicodedata.normalize("NFKD", text)

    # 2️⃣  ligatures（）
    ligatures = {"ﬁ": "fi", "ﬂ": "fl", "ﬀ": "ff", "ﬃ": "ffi", "ﬄ": "ffl"}
    for k, v in ligatures.items():
        text = text.replace(k, v)

    # 3️⃣ 、 ASCII "-"
    text = re.sub(r"[–—−]", "-", text)

    # 4️⃣ 
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")

    # 5️⃣ 
    text = re.sub(r"[\u00A0\u2000-\u200B\u202F\u205F\u3000]", " ", text)

    # 6️⃣ OCR 
    text = text.replace("°", ".")           #   °  .
    text = re.sub(r"\.\s*\.\s*\.", ".", text)  #   ...  . . .  .
    text = re.sub(r"\.{3,}", ".", text)        #   .

    # 7️⃣ （）
    text = re.sub(r"[\x00-\x08\x0b-\x1f\x7f]", "", text)

    # 8️⃣ 
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)

    # 9️⃣ （：A/PV.289）
    text = re.sub(r"A/PV\.\d+", "", text)

    # 🔟 
    text = text.strip()

    return text


# ✅ 
if __name__ == "__main__":
    with open("A_PV-289-EN_ocr_txt_0.txt", "r", encoding="utf-8") as f:
        raw = f.read()

    cleaned = clean_raw_text(raw)

    with open("A_PV-289-EN_cleaned.txt", "w", encoding="utf-8") as f:
        f.write(cleaned)

    print("✅ ！：A_PV-289-EN_cleaned.txt")


✅ 清洗完成！输出文件：A_PV-289-EN_cleaned.txt


In [4]:
import re
from symspellpy import SymSpell, Verbosity
import pkg_resources

def setup_symspell():
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dict_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
    sym_spell.load_dictionary(dict_path, term_index=0, count_index=1)
    return sym_spell


def correct_text_preserve_format(text, sym_spell, whitelist=None):
    """
    （、、）。
    """
    if whitelist is None:
        whitelist = {
            "United", "Nations", "Ethiopia", "Indonesia", "Iran", "Philippines",
            "Security", "Council", "Assembly", "General", "Secretary", "Aklilou",
            "Nasrollah", "Entezam", "New", "York", "President", "Flushing", "Meadow"
        }

    def correct_word(match):
        word = match.group(0)
        # 、、、
        if word in whitelist or word.istitle() or word.isupper() or len(word) <= 2:
            return word
        suggestion = sym_spell.lookup(word.lower(), Verbosity.CLOSEST, max_edit_distance=2)
        if suggestion:
            corrected = suggestion[0].term
            # Note.            if word[0].isupper():
                corrected = corrected.capitalize()
            return corrected
        return word

    # ，
    pattern = re.compile(r"\b[A-Za-z]{3,}\b")
    corrected_text = pattern.sub(correct_word, text)
    return corrected_text


# Note.if __name__ == "__main__":
    sym_spell = setup_symspell()
    with open("A_PV-289-EN_cleaned.txt", "r", encoding="utf-8") as f:
        raw_text = f.read()

    corrected_text = correct_text_preserve_format(raw_text, sym_spell)

    with open("A_PV-289-EN_corrected_preserve.txt", "w", encoding="utf-8") as f:
        f.write(corrected_text)

    print("✅ （），：A_PV-289-EN_corrected_preserve.txt")


✅ 拼写纠错完成（保留标点与格式），输出文件：A_PV-289-EN_corrected_preserve.txt
